<a href="https://colab.research.google.com/github/DAN-BIWOTT/Arasaka-TradingSystems/blob/main/Ensembled_SVR_%26_SFN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install skl2onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.2 which is incompatible.
grpcio-sta

# Import Data Set

In [2]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import re

In [3]:
from google.colab import drive, files
drive.mount('/content/drive') # mount your google colab file

In [6]:
filename_in_drive = "/content/drive/My Drive/Forex/Market Data GBPUSD before 2025.02.28.csv"
data = pd.read_csv(filename_in_drive)
# Version Number
model_version = "2.1"
# Extract date from filename_in_drive
match = re.search(r'\b(\d{4}\.\d{2}\.\d{2})\b', filename_in_drive)
if match:
    extracted_date = match.group(1)
else:
    print("No date found in the filename.")
data.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'Market Data GBPUSD before 2025.02.28'

In [ ]:
num_rows = len(data)
num_rows

In [ ]:
data = data[::-1]
# Reduce the dataset to the first 10,000 rows because
data = data.head(10000)
original_data = data.copy()

data.head(3)

In [ ]:
num_rows = len(data)
num_rows

In [ ]:
data.tail()

In [ ]:
data.info()

In [ ]:
#Define the forecast horizon
look_ahead = 1

In [ ]:
#Calculate the median, which is the midpoint between the high and low prices.
data['Candlestick_Median'] = (data['High'] + data['Low']) / 2

data.head()

In [ ]:
data['Candlestick_Median'] = data['Candlestick_Median'].shift(-1)
data = data.rename(columns={'Candlestick_Median': 'Future_Median'})
data['Price_Difference'] = data['High'] - data['Low']
data['Open_Close_Change_Pct'] = (data['Close'] - data['Open']) / data['Open'] * 100
data['High_Low_Change_Pct'] = (data['High'] - data['Low']) / data['Low'] * 100
data['Volume_MA_20'] = data['Volume'].rolling(window=20).mean() # 20-period moving average of volume
data['Volume_Change_Pct'] = data['Volume'].pct_change() * 100 # Percentage change in volume

In [ ]:
data.head()

In [ ]:
# Create lagged median prices
data['median_t-1'] = data['Future_Median'].shift(2)
data['median_t-2'] = data['Future_Median'].shift(3)

In [ ]:
# Update features (X)
X = data[["Open", "High", "Low", "Close", "Price_Difference", "Open_Close_Change_Pct",
          "High_Low_Change_Pct", "Volume", "Volume_MA_20", "Volume_Change_Pct",
          "median_t-1", "median_t-2"]].copy()
y = data["Future_Median"]  # Target variable

In [ ]:
# Removing NAN Values while preserving time series structure
# Creating a temporary DataFrame combining X and y for consistent indexing
temp_df = pd.concat([X, y], axis=1)

# Dropping rows with NaN values in any column (features or target)
temp_df.dropna(inplace=True)

# Extractting updated X and y from the temporary DataFrame
X = temp_df[X.columns]  # Features
y = temp_df[y.name]  # Target variable

 Ensemble an SVR and an SFN

In [ ]:
from sklearn.svm import LinearSVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
import numpy as np

scale

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print("NaN values in X (features):")
print(X[X.isnull().any(axis=1)])  # Display rows with NaN in X

print("\nNaN values in y (target):")
print(y[y.isnull()])

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 1]}  # Example hyperparameter grid
grid_search = GridSearchCV(LinearSVR(max_iter=10000), param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_linear_svr_model = grid_search.best_estimator_

SFN with Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

# Define the hyperparameter grid for the SFN (MLPRegressor)
param_grid_sfn = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

# Create GridSearchCV object for the SFN
grid_search_sfn = GridSearchCV(MLPRegressor(max_iter=1000, random_state=42),
                               param_grid_sfn, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object to the training data
grid_search_sfn.fit(X_train, y_train)

# Get the best SFN model
best_sfn_model = grid_search_sfn.best_estimator_

print("Best SFN Model:")
print(best_sfn_model)

Temporary SFN without Grid search

In [ ]:
# from sklearn.neural_network import MLPRegressor

# # Define the SFN model with pre-selected hyperparameters
# best_sfn_model = MLPRegressor(
#     hidden_layer_sizes=(100,),  # Example: Using one hidden layer with 100 neurons
#     activation='relu',        # Example: Using ReLU activation function
#     solver='adam',            # Example: Using the Adam optimizer
#     alpha=0.001,             # Example: Setting regularization strength
#     learning_rate='constant', # Example: Using a constant learning rate
#     max_iter=1000,           # Maximum number of iterations
#     random_state=42          # For reproducibility
# )

# # Fit the SFN model to the training data
# best_sfn_model.fit(X_train, y_train)

# print("SFN Model:")
# print(best_sfn_model)

In [ ]:
ensemble_model = VotingRegressor(estimators=[('linear_svr', best_linear_svr_model), ('sfn', best_sfn_model)],
                                 weights=[0.7, 0.3])  # Example: 0.7 for LinearSVR, 0.3 for SFN
ensemble_model.fit(X_train, y_train)

In [ ]:
# Evaluate the ensemble model
y_pred = ensemble_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error of Ensemble: {mse}")

Graph the perfomance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 1. Predictions (Same as before)
y_train_pred = ensemble_model.predict(X_train)
y_test_pred = ensemble_model.predict(X_test)

# 2. Training Data Performance
plt.figure(figsize=(10, 6))
plt.plot(y_train.index, y_train, label='Actual Median', color='blue')  # Actual median in blue
plt.plot(y_train.index, y_train_pred, label='Predicted Median', color='red', linestyle='--')  # Predicted median in dashed red
plt.title('Training Data: Actual vs. Predicted Median Price')
plt.xlabel('Data Point Index')
plt.ylabel('Median Price')
plt.legend()
plt.grid(True)
plt.show()

# 3. Testing Data Performance
plt.figure(figsize=(10, 6))
plt.plot(y_test.index, y_test, label='Actual Median', color='blue')  # Actual median in blue
plt.plot(y_test.index, y_test_pred, label='Predicted Median', color='red', linestyle='--')  # Predicted median in dashed red
plt.title('Testing Data: Actual vs. Predicted Median Price')
plt.xlabel('Data Point Index')
plt.ylabel('Median Price')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import json
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

# 1. Calculate Performance Metrics
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

train_explained_variance = explained_variance_score(y_train, y_train_pred)
test_explained_variance = explained_variance_score(y_test, y_test_pred)

# 2. Store Metrics in a Dictionary
performance_metrics = {
    "train_mse": train_mse,
    "test_mse": test_mse,
    "train_rmse": train_rmse,
    "test_rmse": test_rmse,
    "train_mae": train_mae,
    "test_mae": test_mae,
    "train_r2": train_r2,
    "test_r2": test_r2,
    "train_explained_variance": train_explained_variance,
    "test_explained_variance": test_explained_variance
}

# 3. Get Features and Model Name
features = list(X.columns)
model_name = f"{model_version}_Ensemble Model (Linear SVR + SFN)_{extracted_date}"  # Update model name

# 4. (Optional) Scaling Parameters - If you used scaling, include this part
scaling_parameters = {
    "mean": scaler.mean_.tolist(),
    "std_dev": scaler.scale_.tolist()
}

# 5. Create Log Data Dictionary
log_data = {
    "model_name": model_name,
    "features": features,
    "performance_metrics": performance_metrics,
    # "scaling_parameters": scaling_parameters  # Include if you have scaling parameters
}

# 6. Save to JSON File
# Assuming 'extracted_date' is defined earlier in your code
with open(f"{model_name}_log_{extracted_date}.json", "w") as f:
    json.dump(log_data, f, indent=4)

print(f"{model_version}_Model log saved to {model_name}_log_{extracted_date}.json")

In [ ]:
print("Scaling Mean:", ', '.join(map(str, scaler.mean_)))
print("Scaling Standard Deviation:", ', '.join(map(str, scaler.scale_)))

ONNX file Generation

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx

In [ ]:
initial_type = [('input', FloatTensorType([None, X_test.shape[1]]))]  # Define input shape based on your features

In [ ]:
# Convert weights to a NumPy array before converting to ONNX
ensemble_model.weights = np.array(ensemble_model.weights)

initial_type = [('input', FloatTensorType([None, X_test.shape[1]]))]  # Define input shape based on your features
onnx_model = convert_sklearn(ensemble_model, initial_types=initial_type)
with open(f"{model_version}_ensemble_model_{extracted_date}.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())